# WNV Model - NARR Precipitation Data Analysis
In this notebook we will calculate various indices for the urban counties using the [precipitation dataset from NARR](https://www.esrl.noaa.gov/psd/data/gridded/data.narr.html).

The years we will calculate these values across will be 1999-2015.

The summer period is defined as June 1st to August 1st.

A dry day is defined as less than 0.0393701 inches (1mm) of precipitation.

## Imports


In [251]:
import pandas as pd
import numpy as np
from scipy.stats import entropy
import pysal
import datetime
import os

## 1) Ratio of Peak to Average Rainfall
We will calculate the maximal and mean rainfall over this 61 day period for each county.

In [ ]:
outfile = open('./county_data/ratio.csv', 'w')
writer = csv.writer(outfile, delimiter=',')

# CSV Header
writer.writerow(['Year', 'County Name', 'GEOID', 'Ratio of Max to Mean'])

frame = pd.read_pickle(output_dir + '1999.pkl')
frame['max'] = frame.max(axis=1)
frame['mean'] = frame.mean(axis=1)
frame['ratio'] = round(frame['max'] / frame['mean'], 4)
all_data = (frame.iloc[:,0].to_frame().join(frame.iloc[:,-1]))
cols = all_data.columns.values
cols[-1] = '1999'
all_data.columns = cols

for year in range(2000, 2016):
    frame = pd.read_pickle(output_dir + str(year) + '.pkl')
    
    # Max over 62 days of each county
    frame['max'] = frame.max(axis=1)
    # Mean over 62 days of each county
    frame['mean'] = frame.mean(axis=1)
    #Ratio of Max to Mean
    frame['ratio'] = round(frame['max'] / frame['mean'], 4)
    
    data = (frame.iloc[:,-1].to_frame())
    
    cols = data.columns.values
    cols[-1] = str(year)
    data.columns = cols
    
    all_data = all_data.join(data)

all_data.to_csv('./county_data/ratio.csv')

## 2) Max Ratio

This will create a spreadsheet of the maximum ratios along with the ratios within 5% of the maximum.

In [ ]:
outfile = open('./county_data/max_ratio.csv', 'w')
writer = csv.writer(outfile, delimiter=',')

for year in range(1999, 2016):
    frame = pd.read_pickle(output_dir + str(year) + '.pkl')
    
    # Max over 62 days of each county
    frame['max'] = frame.max(axis=1)
    # Mean over 62 days of each county
    frame['mean'] = frame.mean(axis=1)
    #Ratio of Max to Mean
    frame['ratio'] = round(frame['max'] / frame['mean'], 4)
    
    # Obtain and write max ratio's data
    max_id = frame['ratio'].idxmax()
    max_ratio = frame['ratio'].max()
    name = frame['NAME'].loc[int(max_id)]
    
    if len(str(max_id)) < 5:
        max_id = int('0' + str(max_id))
    writer.writerow([year, name, max_id, max_ratio])

    # Get all ratios within 5% of the max ratio
    diff = max_ratio * 0.05
    max_ratios = frame.loc[(max_ratio - diff < frame['ratio'])]
    for geoid in max_ratios.index:
        if geoid != max_id:
            data = max_ratios.loc[geoid]
            writer.writerow([year, data['NAME'], geoid, data['ratio']])
            
outfile.close()

## 3) Counting Dry Days Over The Period

We will convert the data to two spreadsheets:

1) The count of dry days over the period of June 1st to August 1st

2) The percentage of dry days over the period (dry days / total days)

We define a dry day as 0 inches of rainfall.

In [ ]:
# Create a base frame with Year 1999
frame = pd.read_pickle(output_dir + '1999.pkl')
frame.index = frame.index.apply(lambda x: x if len(x) < 6 )
zeroes = frame.isin([0])
zeroes = zeroes.sum(1)
zeroes = zeroes.to_frame()

# Change column to 1999
cols = zeroes.columns.values
cols[0] = '1999'
zeroes.columns = cols

# Create the dataframe
data = frame.iloc[:,:1].join(zeroes)
# print(data)

for year in range(2000, 2016):
    frame = pd.read_pickle(output_dir + str(year) + '.pkl')
    zeroes = frame.isin([0])
    zeroes = zeroes.sum(1)
    zeroes = zeroes.to_frame()
    
    cols = zeroes.columns.values
    cols[0] = str(year)
    zeroes.columns = cols
    
    data = data.join(zeroes)

num_days = len(frame.columns[1:])

percent_data = data.iloc[:,1:]
percent_data = percent_data.apply(lambda x: round(x / num_days * 100, 4))
percent_data = frame['NAME'].to_frame().join(percent_data)


percent_data.to_csv('./county_data/dry_days_percentage.csv')
data.to_csv('./county_data/dry_days.csv')

## 4) Shannon's Entropy

We will calculate Shannon's entropy over this 62 day period for each year. 

Since Shannon's entropy requires a discrete distribution, we will bin the data into 10 bins before calculating the entropy.


In [ ]:
outfile = open('./county_data/shannons_entropy.csv', 'w')
writer = csv.writer(outfile, delimiter=',')
writer.writerow(['Year', 'Dry 8 Bins', 'Dry 12 Bins', 'Wet 8 Bins', 'Wet 12 Bins'])

for year in range(1999, 2016):
    frame = pd.read_pickle(output_dir + str(year) + '.pkl')
    
    num_days = len(frame.columns[1:])
    
    probability_frame = (frame.isin([0]).sum(1).to_frame())
    
    dry_frame = frame.isin([0]).sum(1).to_frame()  # Number of Dry Days (count of 0's)
    wet_frame = num_days-dry_frame  # Number of wet days (total days - dry days)
    
    dry_prob = probability_frame.apply(lambda x: (x / num_days))  # Probability of a dry day
    wet_prob = probability_frame.apply(lambda x: 1 - (x/num_days))  
    
    #Binning Dry Probability - 8 Bins
    dry_prob['bin'] = pd.cut(dry_prob.iloc[:,0], 8)
    dry_prob_bin = dry_prob.groupby(dry_prob['bin']).size()

    #Binning Wet Probability - 8 Bins
    wet_prob['bin'] = pd.cut(wet_prob.iloc[:,0],8)
    wet_prob_bin = wet_prob.groupby(wet_prob['bin']).size()
    
    # Binning Dry Days - 8 Bins
    dry_frame['bin'] = pd.cut(dry_frame.iloc[:,0],8)
    dry_days_bin = dry_frame.groupby(dry_frame['bin']).size()
    
    # Binning Wet Days - 8 Bins
    wet_frame['bin'] = pd.cut(wet_frame.iloc[:,0],8)
    wet_days_bin = wet_frame.groupby(wet_frame['bin']).size()
    
    
    # Repeating with 12 bins (one for each month of the year)
    
    dry_prob = probability_frame.apply(lambda x: (x / num_days))  # Probability of a dry day
    wet_prob = probability_frame.apply(lambda x: 1 - (x/num_days))  
    
    #Binning Dry Probability - 12 Bins
    dry_prob['bin'] = pd.cut(dry_prob.iloc[:,0], 12)
    dry_prob_bin_12 = dry_prob.groupby(dry_prob['bin']).size()

    #Binning Wet Probability - 12 Bins
    wet_prob['bin'] = pd.cut(wet_prob.iloc[:,0],12)
    wet_prob_bin_12 = wet_prob.groupby(wet_prob['bin']).size()
    
    # Binning Dry Days - 12 Bins
    dry_frame['bin'] = pd.cut(dry_frame.iloc[:,0],12)
    dry_days_bin_12 = dry_frame.groupby(dry_frame['bin']).size()
    
    # Binning Wet Days - 12 Bins
    wet_frame['bin'] = pd.cut(wet_frame.iloc[:,0],12)
    wet_days_bin_12 = wet_frame.groupby(wet_frame['bin']).size()
    
    
#     print(entropy(wet_days_bin), entropy(wet_prob_bin), entropy(dry_days_bin), entropy(dry_prob_bin))
    
    writer.writerow([year, entropy(dry_days_bin), entropy(dry_days_bin_12), entropy(wet_days_bin), entropy(wet_days_bin_12)])
outfile.close()


## 5) Gini and Theil Coefficients for Summer Data

In [ ]:
for year in range(1999, 2016):
    
    frame = pd.read_pickle('./county_data/dataframes/summer_data/' + str(year) + '.pkl')
    frame = frame.reset_index()    
    name = frame.iloc[:,1]
    frame = frame.drop('NAME', axis=1)

    if year == 1999:        
        gini = pd.DataFrame(index=frame.index)
        gini = gini.join(name)
        
        theil = pd.DataFrame(index=frame.index)
        theil = theil.join(name)
    
    gini_frame = frame.apply(pysal.explore.inequality.gini.Gini, axis=1).apply(lambda x: x.g)
    theil_frame = frame.apply(pysal.explore.inequality.theil.Theil, axis=1).apply(lambda x: x.T)
    
    gini = gini.join(gini_frame.to_frame(str(year)))
    theil = theil.join(theil_frame.to_frame(str(year)))
       
gini.set_index(frame['GEOID'], drop=True, inplace=True)
theil.set_index(frame['GEOID'], drop=True, inplace=True)

gini.to_csv('./county_data/gini/gini_index_summer.csv')
theil.to_csv('./county_data/theil/theil_index_summer.csv')

## 6) Gini and Theil Coefficient over Entire Dataset
We will calculate the Gini and Theil coefficient for each county over each year using [pysal's implementation](https://pysal.readthedocs.io/en/v1.12.0/library/inequality/index.html).

In [ ]:
import pysal
import numpy as np

### Calculating the Coefficients on Unfiltered Data

In [ ]:
for year in range(1999, 2016):
    frame = pd.read_pickle('./county_data/dataframes/all_data/' + str(year) + '.pkl')  # Load the data
    name = frame.iloc[:,0]
    data = frame.iloc[:,1:]  # Ignore the NAME column
    
    if year == 1999:
        gini = pd.DataFrame(index=frame.index)
        gini = gini.join(name)
        
        theil = pd.DataFrame(index=frame.index)
        theil = theil.join(name)

    gini_data = data.apply(pysal.explore.inequality.gini.Gini, axis=1).apply(lambda x: x.g)
    theil_data = data.apply(pysal.explore.inequality.theil.Theil, axis=1).apply(lambda x: x.T)
    
    
    gini = gini.join(gini_data.to_frame(str(year)))
    theil = theil.join(theil_data.to_frame(str(year)))
  

gini.to_csv('./county_data/gini/gini_index.csv')
theil.to_csv('./county_data/theil/theil_index.csv')

### Filtered Data

In [ ]:
for year in range(1999, 2016):
    frame = pd.read_pickle('./county_data/dataframes/filtered_freezing_data/' + str(year) + '.pkl')  # Load the data
    data = frame.iloc[:,1:]  # Ignore the NAME column
    
    if year == 1999:
        gini = pd.DataFrame(index=frame.index)
        gini = gini.join(name)
        
        theil = pd.DataFrame(index=frame.index)
        theil = theil.join(name)
    
    gini_data = data.apply(pysal.explore.inequality.gini.Gini, axis=1).apply(lambda x: x.g)
    theil_data = data.apply(pysal.explore.inequality.theil.Theil, axis=1).apply(lambda x: x.T)
    
    gini = gini.join(gini_data.to_frame(str(year)))
    theil = theil.join(theil_data.to_frame(str(year)))
    
    
gini.to_csv('./county_data/gini/gini_index_filtered.csv')
theil.to_csv('./county_data/theil/theil_index_filtered.csv')

## 7) Aggregated Precipitation

We will calculate the total precipitation (both filtered and unfiltered) across each year for each county.



In [253]:
out_dir = './county_data/aggregated/'
if not os.path.exists(out_dir):
    print('Making', out_dir)
    os.makedirs(out_dir)

#### Unfiltered Aggregation

In [ ]:
frame = pd.read_pickle('./county_data/dataframes/all_data/1999.pkl').set_index('GEOID', drop=True)
name = frame.iloc[:, 0]
frame = frame.iloc[:,1:].sum(axis=1)
frame = frame.to_frame('1999')

for year in range(2000,2016):
    sub_frame = pd.read_pickle('./county_data/dataframes/all_data/' + str(year) + '.pkl').set_index('GEOID', drop=True)
    sub_frame = sub_frame.sum(axis=1).rename(str(year))
    frame = frame.join(sub_frame)

frame = name.to_frame().join(frame)
frame.to_csv('./county_data/aggregated/unfiltered_aggregated_precipitation.csv')

#### Filtered Aggregation

In [ ]:
frame = pd.read_pickle('./county_data/dataframes/filtered_freezing_data/1999.pkl')
name = frame.iloc[:, 0]
frame = frame.iloc[:,1:].sum(axis=1)
frame = frame.to_frame('1999')

for year in range(2000,2016):
    sub_frame = pd.read_pickle('./county_data/dataframes/filtered_freezing_data/' + str(year) + '.pkl')
    sub_frame = sub_frame.sum(axis=1).rename(str(year))
    frame = frame.join(sub_frame)
    
frame = name.to_frame().join(frame)
frame.to_csv('./county_data/aggregated/filtered_aggregated_precipitation.csv')

## 8) Count of Total Dry Days

We consider a dry day to be where the precipitation is less than 0.0393701 inches (1 millimeter).

#### Using Filtered Dataset

In [ ]:
frame = pd.read_pickle('./county_data/dataframes/filtered_freezing_data/1999.pkl')
name = frame.iloc[:,0].to_frame('NAME')
data = frame.iloc[:,1:]  # Ignore the NAME column

dry_days = data < 0.0393701  # True if Dry, False if Wet
dry_days = dry_days.applymap(lambda x: -1 if x is False else 1)  # Replace false with -1, true with 1

data = data * dry_days  # Wet days will have a negative value
data = data.applymap(lambda x: np.nan if x < 0 else x)  # Wet days and freezing days will be NaN

# Count the number of remaining values across each row
frame = data.count(axis=1).to_frame('1999')

frame = name.join(frame)

for year in range(2000, 2016):
    subframe = pd.read_pickle('./county_data/dataframes/filtered_freezing_data/' + str(year) + '.pkl')
    data = subframe.iloc[:,1:]  # Ignore the NAME column
    dry_days = data < 0.0393701  # True if Dry, False if Wet
    
    dry_days = dry_days.applymap(lambda x: -1 if x is False else 1)  # Replace false with -1, true with 1
    
    data = data * dry_days  # Wet days will have a negative value
    data = data.applymap(lambda x: np.nan if x < 0 else x)  # Wet days and freezing days will be NaN
    
    # Count the number of remaining values across each row
    data = data.count(axis=1).to_frame(str(year))
    frame = frame.join(data)
    
frame.to_csv('./county_data/dry/dry_days_filtered.csv')
print(frame)

#### Using Unfiltered Dataset

In [ ]:
frame = pd.read_pickle('./county_data/dataframes/all_data/1999.pkl')
name = frame.iloc[:,0].to_frame('NAME')
data = frame.iloc[:,1:]  # Ignore the NAME column

dry_days = data < 0.0393701  # True if Dry, False if Wet
dry_days = dry_days.applymap(lambda x: -1 if x is False else 1)  # Replace false with -1, true with 1

data = data * dry_days  # Wet days will have a negative value
data = data.applymap(lambda x: np.nan if x < 0 else x)  # Wet days and freezing days will be NaN

# Count the number of remaining values across each row
frame = data.count(axis=1).to_frame('1999')
frame = name.join(frame)

for year in range(2000, 2016):
    subframe = pd.read_pickle('./county_data/dataframes/all_data/' + str(year) + '.pkl')
    data = subframe.iloc[:,1:]  # Ignore the NAME column
    dry_days = data < 0.0393701  # True if Dry, False if Wet
    
    dry_days = dry_days.applymap(lambda x: -1 if x is False else 1)  # Replace false with -1, true with 1
    
    data = data * dry_days  # Wet days will have a negative value
    data = data.applymap(lambda x: np.nan if x < 0 else x)  # Wet days and freezing days will be NaN
    
    # Count the number of remaining values across each row
    data = data.count(axis=1).to_frame(str(year))
    frame = frame.join(data)
    
frame.to_csv('./county_data/dry/dry_days_unfiltered.csv')
print(frame)

## 9) Maximum Consecutive Dry Days

We will find the longest chain of consecutive dry days for each county over each year.

We consider a dry day to be where the precipitation is less than 0.0393701 inches (1 millimeter).

In [ ]:
def consecutive_days(row):
#     num_days = 3
    consecutive = row * (row.groupby((row != row.shift()).cumsum()).cumcount()+1)
    return consecutive.max()

In [ ]:
name = pd.read_pickle('./county_data/dataframes/filtered_freezing_data/1999.pkl').iloc[:,0].to_frame()

for year in range(1999, 2016):
    frame = pd.read_pickle('./county_data/dataframes/filtered_freezing_data/' + str(year) + '.pkl')
    data = frame.iloc[:,1:]
    dry_days = data < 0.0393701  # True if Dry, False if Wet
    
    consecutive = dry_days.apply(consecutive_days, axis=1).to_frame(str(year))
    name = name.join(consecutive)

name.to_csv('./county_data/dry/max_consecutive_dry_days.csv')

## 10) Gini and Theil Inequality on Weekly Aggregated Data

We will sum the precipitation across each week, and calculate the gini and theil inequality.

In [249]:
first_day = {
             1999:'Thu', 2000:'Sat', 2001:'Sun', 2002:'Mon', 2003:'Tue', 
             2004:'Wed', 2005:'Fri', 2006:'Sat', 2007:'Sun', 2008:'Mon', 
             2009:'Wed', 2010:'Thu', 2011:'Fri', 2012:'Sat', 2013:'Mon',
             2014:'Tue', 2015:'Wed',
            }

for year in range(1999, 2016):    
    frame = pd.read_pickle('./county_data/dataframes/all_data/datetime_headers/' + str(year) + '.pkl')
    name = frame.iloc[:, 0]

    if year == 1999:
        gini = pd.DataFrame(index=frame.index)
        gini = gini.join(name)
        
        theil = pd.DataFrame(index=frame.index)
        theil = theil.join(name)
        
    data = frame.iloc[:,2:]

    # Pickling the frame doesn't preserve column datatype (datetime), so we have to reconvert anyways
    data.columns = pd.to_datetime(data.columns)
    
    weekly = data.resample('W-'+first_day[year], axis=1).sum()

    gini_data = weekly.apply(pysal.explore.inequality.gini.Gini, axis=1).apply(lambda x: x.g).rename(year)
    theil_data = weekly.apply(pysal.explore.inequality.theil.Theil, axis=1).apply(lambda x: x.T).rename(year)
    
    gini = gini.join(gini_data)
    theil = theil.join(theil_data)
    
gini.to_csv('./county_data/gini/gini_weekly.csv')
theil.to_csv('./county_data/theil/theil_weekly.csv')